2D 탑뷰로 구현하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from scipy.signal import savgol_filter

# ---------- 경로 / 파라미터 ----------
CSV_PATH = "/content/drive/MyDrive/Little_kid_0912/result_csv/pitch_coordinate_log.csv"
OUT_PATH = "/content/drive/MyDrive/Little_kid_0912/result/topdown_tracking_court_smoothed.mp4"

COURT_W, COURT_H = 10.0, 20.0     # 반코트 10x20m
FPS = 30                           # 최종 영상 FPS (CSV time_s가 있으면 거기에 맞춰도 OK)
FIGSIZE = (6, 12)                  # matplotlib in inches
DPI = 200                          # 해상도 스케일
FRAME_SIZE = (int(FIGSIZE[0]*DPI), int(FIGSIZE[1]*DPI))  # (W,H) in px

# 공 스무딩 설정
MAX_M_PER_FRAME = 0.6              # 프레임당 허용 이동(m) 초과분은 NaN으로 날리고 보간(공중/튐 억제)
SG_WINDOW = 9                       # Savitzky–Golay window(홀수)
SG_POLY = 2                         # Savitzky–Golay poly order
TRAIL = 40                          # 그려줄 궤적 길이(프레임 수)

# ---------- 데이터 로드/분리 ----------
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["court_x_m","court_y_m"])  # 좌표 없는 행 제거

players = df[df["class"] == "player"].copy()
ball    = df[df["class"] == "ball"].copy()

max_frame = int(df["frame"].max())

# ---------- 공 궤적: 스파이크 제거 → 선형보간 → Savitzky–Golay ----------
# 1) 프레임 기준으로 정렬/리인덱스
ball = ball.sort_values("frame")
ball_full = pd.DataFrame({"frame": np.arange(max_frame+1)})
ball_full = ball_full.merge(ball[["frame","court_x_m","court_y_m"]], on="frame", how="left")

# 2) 1차 선형 보간(빈 프레임 채움)
ball_full[["court_x_m","court_y_m"]] = ball_full[["court_x_m","court_y_m"]].interpolate(limit_direction="both")

# 3) 속도(프레임당 이동) 계산해서 스파이크 컷
dx = ball_full["court_x_m"].diff()
dy = ball_full["court_y_m"].diff()
disp = np.sqrt((dx.fillna(0))**2 + (dy.fillna(0))**2)

spike_mask = disp > MAX_M_PER_FRAME
ball_full.loc[spike_mask, ["court_x_m","court_y_m"]] = np.nan

# 4) 다시 보간
ball_full[["court_x_m","court_y_m"]] = ball_full[["court_x_m","court_y_m"]].interpolate(limit_direction="both")

# 5) Savitzky-Golay 스무딩 (유효한 길이일 때만)
def sg_smooth(arr, win, poly):
    arr = arr.astype(float)
    if np.sum(~np.isnan(arr)) < max(win, poly+2):
        return arr
    ok = ~np.isnan(arr)
    tmp = arr.copy()
    tmp[ok] = savgol_filter(tmp[ok], window_length=win if np.sum(ok) >= win else (np.sum(ok)//2)*2+1,
                            polyorder=min(poly, max(1, (np.sum(ok)-1)//2)))
    return tmp

ball_full["sx"] = sg_smooth(ball_full["court_x_m"].values, SG_WINDOW, SG_POLY)
ball_full["sy"] = sg_smooth(ball_full["court_y_m"].values, SG_WINDOW, SG_POLY)

# ---------- 코트 그리기 함수 ----------
def draw_half_futsal(ax):
    # 외곽
    ax.add_patch(patches.Rectangle((0,0), COURT_W, COURT_H, fill=False, lw=2, color="black"))
    # 하프라인
    ax.plot([0, COURT_W], [COURT_H/2, COURT_H/2], linestyle="--", linewidth=1, color="gray")
    # 센터 서클 (R=3m)
    center = (COURT_W/2, COURT_H/2)
    ax.add_patch(patches.Circle(center, radius=3.0, fill=False, lw=1, linestyle=":"))
    # 골(폭 3m) – 골라인(y=0) 중앙에 표시
    goal_w = 3.0
    gx1, gx2 = center[0]-goal_w/2, center[0]+goal_w/2
    ax.plot([gx1, gx2], [0, 0], lw=4, color="black")  # 골 라인 두껍게
    # 패널티 아크(반원, R=6m, 골 중앙 기준)
    pen_r = 6.0
    arc = patches.Arc((center[0], 0), width=2*pen_r, height=2*pen_r,
                      theta1=0, theta2=180, lw=1.2, linestyle="-.")  # 위쪽 반원
    ax.add_patch(arc)
    # 눈금/라벨
    ax.set_xlim(0, COURT_W)
    ax.set_ylim(0, COURT_H)
    ax.set_aspect("equal", adjustable="box")
    ax.set_xlabel("Court X (m)")
    ax.set_ylabel("Court Y (m)")

# ---------- 색상/마커 준비 ----------
player_ids = players["oid"].unique().tolist()
cmap = plt.get_cmap("tab10")
pid2col = {pid: cmap(i % 10) for i, pid in enumerate(sorted(player_ids))}

# ---------- 비디오 작성 ----------
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUT_PATH, fourcc, FPS, FRAME_SIZE)

for fi in range(max_frame + 1):
    fig, ax = plt.subplots(figsize=FIGSIZE, dpi=DPI)
    draw_half_futsal(ax)

    # 플레이어: 각자 TRAIL 길이만큼 궤적
    for pid in player_ids:
        sub = players[(players["oid"] == pid) & (players["frame"] <= fi)].tail(TRAIL)
        if len(sub):
            ax.plot(sub["court_x_m"], sub["court_y_m"], "-", lw=2, color=pid2col[pid], label=f"P{pid}")
            ax.plot(
                sub["court_x_m"].iloc[-1], sub["court_y_m"].iloc[-1],
                "o", ms=12, color=pid2col[pid], markeredgecolor="white", markeredgewidth=1.2, zorder=5
            )

    # 공: 스무딩된 좌표 사용 (원한다면 원본도 얇게 점선으로 겹쳐볼 수 있음)
    sub_idx = np.arange(max(0, fi-TRAIL+1), fi+1)
    bx = ball_full.loc[sub_idx, "sx"].values
    by = ball_full.loc[sub_idx, "sy"].values

    # 원본(옵션) — 희미하게
    # bx_raw = ball_full.loc[sub_idx, "court_x_m"].values
    # by_raw = ball_full.loc[sub_idx, "court_y_m"].values
    # ax.plot(bx_raw, by_raw, color="red", alpha=0.25, lw=1, linestyle=":")

    ax.plot(bx, by, "r-", lw=2.5, label="Ball")
    ax.plot(bx[-1], by[-1], "o", ms=6, color="red")

    ax.set_title(f"Top-Down Court Tracking (Frame {fi})", pad=10)
    # 범례 너무 많으면 생략하거나 위치 조정
    # ax.legend(loc="upper right", ncol=2, fontsize=8)

    # Matplotlib → OpenCV (RGBA→BGR)
    fig.canvas.draw()
    buf = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    w, h = fig.canvas.get_width_height()
    img = buf.reshape(h, w, 4)[:, :, :3]                  # RGB
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)            # BGR for OpenCV
    # 크기 보정(혹시 사이즈 불일치 시)
    if (img.shape[1], img.shape[0]) != FRAME_SIZE:
        img = cv2.resize(img, FRAME_SIZE, interpolation=cv2.INTER_AREA)

    out.write(img)
    plt.close(fig)

out.release()
print("✅ Saved:", OUT_PATH)

✅ Saved: /content/drive/MyDrive/Little_kid_0912/result/topdown_tracking_court_smoothed.mp4
